# Matching mechanisms for kidney tranplantations - Dentan & Zhioua

In [1]:
##Importations
import numpy as np
from time import time
import random

In [2]:
#Paramètres globaux de l'algorithme (graphe de l'exemple 4.3)
#On prend n=13 et non n=12 pour prendre en compte la file d'attente
n=13

#Liste des préférences (on suppose que les patients sont déja triés par ordre de priorité.
#0 représente la liste d'attente, à laquelle on affecte une liste fictive.
P = [[],
     [9,10,1],
     [11,3,5,6,2],
     [2,4,5,6,7,8,0],
     [5,9,1,8,10,3,0],
     [3,7,11,4,5],
     [3,5,8,6],
     [6,1,3,9,10,0],     #erreur d'énoncé pour cette ligne ?? (k1 en trop)
     [6,4,11,2,3,8],
     [3,11,0],
     [11,1,4,5,6,7,0],
     [3,6,5,11],
     [11,3,9,8,10,12]]

#représentation du graphe sous la forme d'une matrice d'adjacence.
#la 0-ème ligne encode la liste d'attente. La 0-ème ligne vaut 1 (la liste d'attente accepte tout le monde)
#K[i,j] = 1 ssi le patient i accepte le rein j

K = np.zeros(shape=(n,n),dtype=int) 
for i in range(1,n):
    for j in range(len(P[i])):   
       K[i,P[i][j]]=1

for i in range(n):
    K[0,i] = 1


In [3]:
print(K)

[[1 1 1 1 1 1 1 1 1 1 1 1 1]
 [0 1 0 0 0 0 0 0 0 1 1 0 0]
 [0 0 1 1 0 1 1 0 0 0 0 1 0]
 [1 0 1 0 1 1 1 1 1 0 0 0 0]
 [1 1 0 1 0 1 0 0 1 1 1 0 0]
 [0 0 0 1 1 1 0 1 0 0 0 1 0]
 [0 0 0 1 0 1 1 0 1 0 0 0 0]
 [1 1 0 1 0 0 1 0 0 1 1 0 0]
 [0 0 1 1 1 0 1 0 1 0 0 1 0]
 [1 0 0 1 0 0 0 0 0 0 0 1 0]
 [1 1 0 0 1 1 1 1 0 0 0 1 0]
 [0 0 0 1 0 1 1 0 0 0 0 1 0]
 [0 0 0 1 0 0 0 0 1 1 1 1 1]]


## Question 1

L'algorithme de donation directe ne prend pas en compte la liste des préférences des patients. On peut donc utiliser directement la matrice d'adjacence pour tester en temps constant la propriété "$k_i \in K_i$". En voici le code :

In [4]:
def direct_donation(K):
    assign=[0] #On affecte la liste d'attente à elle-même
    for i in range(1,n):
        if K[i,i]==1:
            assign.append(i)
        else :
            assign.append(0)
    return(assign)

direct_donation(K)

[0, 1, 2, 0, 0, 5, 6, 0, 8, 0, 0, 11, 12]

## Question 2

L'algorithme glouton consiste à parcourir l'ensemble des patients par ordre décroissant de priorité (donc ici en partant du premier patient vu que les patients sont déja triés par ordre de priorité). Pour chaque paire $(k_i,t_i)$ que l'on considère, on affecte le rein $k_p$ le plus haut placé dans la liste des préférences de $t_i$ encore non affecté et tel que l'arrête $(k_i,t_i),(k_p,t_p)$ existe dans le graphe.

In [5]:
def greedy_matching(K,P):
    assigned=n*[False] #sert à vérifier si les reins sont affectes
    matching=n*[0]
    
    for i in range(1,n):
        if not(assigned[i]):
            k=0 #indice qui parcourt la liste des préférences
            found = False
            while not(found) and k<len(P[i]):
                if K[i,P[i][k]]*K[P[i][k],i]==1 and not(assigned[P[i][k]]):
                    found=True
                    matching[i]=P[i][k]
                    matching[P[i][k]]=i
                    assigned[P[i][k]]=True
                    assigned[i]=True
                    if P[i][k] == 0:
                        assigned[0] = False
                        matching[0] = 0
                k+=1
    return matching

greedy_matching(K,P)

[0, 10, 3, 2, 5, 4, 8, 0, 6, 0, 1, 11, 12]

## Question 3

On suppose qu'il n'y a pas de cycle.   
Considérons une paire $(k_i,t_i)$. Lorsque l'on construit la liste $l$ des successeurs de  $(k_i,t_i)$, à chaque étape, on ajoute un élément qui n'est pas déja présent dans $l$ (sinon on crée un cycle). Comme le nombre de sommets est fini on doit nécessairement ajouter $w$ avant d'épuiser l'ensemble des sommets disponibles. Ainsi, $(k_i,t_i)$ est bien la queue d'une $w$-chaîne, ce qui conclut la démonstration.

## Question 4

### **Fonctions générales**

In [6]:
def next_unassignedUnseen(assigned, seen,n):
    """Renvoie le prochain sommet non affecté et non vu.
    L'entrée est constituée de deux listes de booléens de taille n.
    La sortie est un entier, et vaut -1 si tous les sommets ont étés assignés et vus."""
    for k in range(1,n):
        if not assigned[k] and not seen[k]:
            return k
    return (-1)

def detect_cycle_wChains(graph,assigned,n):
    """Fonction qui renvoit :
        -: (0,sCycle) s'il existe un cycle, et sCycle est un élément du cycle ;
        
        -: (1,wChains) s'il n'existe pas de cycle, où wChains est un set contenant l'ensemble des 
        w chaines sous la forme (startOfAWChain, length, highestPriorityPatient). 
        NB : on renvoit toutes les w-chaines, donc si une chaine k1,t1,k2,t2.. est renvoyée, on renvoie
        aussi k2,t2,... (cela est nécessaire pour la règle B) ;
        
        -: (1,set()) si tous les éléments ont étés visités. """
    
    seen = n*[False] # this variable aims at testing each point, but it's not useful to detect a cycle
    s = next_unassignedUnseen(assigned, seen, n)
    
    wChains = set()
    
    
    while s != -1:
        sVar = s # This variable will run throught the hypothetic cycle or w chain
        
        seenInThisCycle = set() # To know if a cycle is found
        length = 0 # To know the length if it's a w-chain
        highestPriority = s
    
        # Going throught the chain
        while sVar!=0 and sVar not in seenInThisCycle: 
            seenInThisCycle.add(sVar)
            length +=1
            highestPriority = min(highestPriority,sVar)
            
            sVar = graph[sVar]
        
        # Result
        if sVar in seenInThisCycle: # There's a cycle !
            return (0,sVar)
        
        elif sVar==0:   # No cycle... let's memorize this w-chain
                        # A possible opimization : memorize all the w-chains we know here !
            wChains.add((s,length,highestPriority))
        
        
        seen[s] = True
        s = next_unassignedUnseen(assigned, seen,n)
    
    return (1,wChains)       

In [7]:
def action_cycle(graph,assigned,matching,kidneyUsed,tuppleCycle,n):
    """Effectue les actions nécessaires lors de la détection d'un cycle.
    tuppleCycle = (0,sCycle) où sCycle représente un sommet appartenant à un cycle, potentiellement de taille 1.
    Cette fonction doit être appelée uniquement à la suite de detect_cycle_wChains."""
    (a,sCycle) = tuppleCycle
    if a!= 0:
        raise ValueError('Fatal error.')
        
    cycleSummits = {sCycle} # Dictionnaire contenant les sommets du cycle
    sVar = graph[sCycle]
    matching[sCycle] = sVar
    assigned[sCycle] = True
    kidneyUsed[sCycle] = True
    
    while sVar!=sCycle:
        cycleSummits.add(sVar)
        matching[sVar] = graph[sVar]
        assigned[sVar] = True
        kidneyUsed[sVar] = True
        
        sVar = graph[sVar]
    
    # Updating the graph
    for k in range(1,n):
        if not assigned[k] and graph[k] in cycleSummits:
            mostPreferedIndex = 0
            while kidneyUsed[P[k][mostPreferedIndex]]: # the wainting list is never assigned so there is an end...
                mostPreferedIndex +=1
            graph[k]= P[k][mostPreferedIndex]

In [8]:
def action_wChain(graph,assigned,matching,kidneyUsed,bestWChain,n):
    """Effectue les actions nécessaires en l'absence de cycles.
    Doit être appellée après get_bestWChains_A ou get_bestWChains_B"""
    
    (s,l,h) = bestWChain
    
    kidneysUsedByTheChain = set()
    
    sVar = graph[s]
    
    matching[s] = sVar
    assigned[s] = True
    kidneyUsed[s] = False # The first kidney is not used
    graph[s] = 0 # The kiney is not assigned : we do so to remember that a w-chain going here have to stop
    
    while sVar!=0:
        if not assigned[sVar]: # Sometime, the kidney is available but the patient is already assigned
            kidneysUsedByTheChain.add(sVar)
            matching[sVar] = graph[sVar]
            assigned[sVar] = True
            kidneyUsed[sVar] = True
        else: # We found an assigned patient : we stop the w-chain
            kidneysUsedByTheChain.add(sVar)
            kidneyUsed[sVar] = True
            break
            
        
        sVar = graph[sVar]
    
    # Updating the graph
    for k in range(1,n):
        if not assigned[k] and graph[k] in kidneysUsedByTheChain:
            mostPreferedIndex = 0
            while kidneyUsed[P[k][mostPreferedIndex]]: # the wainting list is never assigned so there is an end...
                mostPreferedIndex +=1
            graph[k]= P[k][mostPreferedIndex]
    

### **Règle A**

In [9]:
def get_bestWChains_A(graph,assigned,tuppleWChains,n):
    """Renvoie la meilleure w-chaine pour la règle A en l'absence de cycle.
    tuppleCycle = (1,wChains) est le résultat de detect_cycle_wChains.
    Cette fonction doit être appelée uniquement à la suite de detect_cycle_wChains.
    Cette fonction ne doit pas être appellée s'il n'y a pas de w-chaines."""
    (a,wChains) = tuppleWChains
    if a != 1:
        raise ValueError('Fatal error.')
    if len(wChains) == 0:
        raise ValueError('Empty w-chains set.')
    
    # Let's find the longest w-chain
    longestWChainSet = set()
    lengthOfTheChampion = 0
    highestPriorityOfTheChampion = n
    
    for (s,length,highestPriority) in wChains:
        if length > lengthOfTheChampion:
            longestWChainSet = {(s,length,highestPriority)}
            lengthOfTheChampion = length
            highestPriorityOfTheChampion = highestPriority
        
        elif length == lengthOfTheChampion:
            if highestPriority < highestPriorityOfTheChampion:
                longestWChainSet = {(s,length,highestPriority)}
                lengthOfTheChampion = length
                highestPriorityOfTheChampion = highestPriority
                highestPriorityOfTheChampion = highestPriority
            elif highestPriority == highestPriorityOfTheChampion:
                longestWChainSet.add((s,length,highestPriority))
    
    
    # Now, we have the longest w-chains, with the longest highest priorities
    # We have to know all the priorities to rank the w-chains
    
    bestWChain = (0,0,0)
    bestWChainPriorities = [n]
    for (s,length,highestPriority) in longestWChainSet:
        priorities = []
        sVar = s
        while sVar != 0:
            priorities.append(sVar)
            sVar = graph[sVar]
        priorities.sort()
        
        if priorities < bestWChainPriorities:
            bestWChain = (s,length,highestPriority)
            bestWChainPriorities = priorities
    
    return bestWChain

In [10]:
def match_A(P,n,progress=False):
    """Renvoie le matching correspondant à la règle A.
    Si progress==True, on affiche ce qui se passe étape après étape."""
    graph = n*[0]
    assigned = n*[False]
    kidneyUsed = n*[False]
    matching = n*[0]
    
    # Building the graph
    for k in range(1,n):
        graph[k]= P[k][0]
        
    # To print the progress
    step = 0
    if progress:
        print("\n\n=============START PROCESS=============\n\n")
        print("STEP : {}\nGraph      = {}\nAssigned   = {}\nKidneyUsed = {}\nMatching   = {}\n\n".format(step,graph,assigned,kidneyUsed,matching))
    
    # Initialisation    
    detectionResult = detect_cycle_wChains(graph,assigned,n)
    
    while detectionResult != (1,set()):
        if detectionResult[0] == 0: # Cycle
            action_cycle(graph,assigned,matching,kidneyUsed,detectionResult,n)
            if progress:
                print("DETECTED : Cycle\nRule : A\nCycle beginning : {}\n\n".format(detectionResult[1]))
        else:
            bestWChain = get_bestWChains_A(graph,assigned,detectionResult,n)
            action_wChain(graph,assigned,matching,kidneyUsed,bestWChain,n)
            if progress:
                print("DETECTED : w-Chain\nRule : A\nChain beginning : {}\nChain length : {}\n\n".format(bestWChain[0],bestWChain[1]))
        
        step += 1
        if progress:
                        print("STEP : {}\nGraph      = {}\nAssigned   = {}\nKidneyUsed = {}\nMatching   = {}\n\n".format(step,graph,assigned,kidneyUsed,matching))
        
        
        detectionResult = detect_cycle_wChains(graph,assigned,n)
    
    
    if progress:
        print("==============END PROCESS==============\n\n\n\n")
    return matching
    

In [11]:
match_A(P,n)

[0, 9, 11, 2, 8, 7, 5, 6, 4, 0, 1, 3, 10]

### Règle B

In [12]:
def get_bestWChains_B(graph,assigned,tuppleWChains,n):
    """Renvoie la meilleure w-chaine pour la règle B en l'absence de cycle.
    tuppleCycle = (1,wChains) est le résultat de detect_cycle_wChains.
    Cette fonction doit être appelée uniquement à la suite de detect_cycle_wChains.
    Cette fonction ne doit pas être appellée s'il n'y a pas de w-chaines."""
    (a,wChains) = tuppleWChains
    if a != 1:
        raise ValueError('Fatal error.')
    if len(wChains) == 0:
        raise ValueError('Empty w-chains set.')
    
    # Let's find the best w-chain
    bestWChain = (n,n,n)
    
    for (s,length,highestPriority) in wChains:
        if s < bestWChain[0]:
            bestWChain = (s,length,highestPriority)
    
    return bestWChain

In [13]:
def match_B(P,n,progress=False):
    """Renvoie le matching correspondant à la règle B.
    Si progress==True, on affiche ce qui se passe étape après étape."""
    graph = n*[0]
    assigned = n*[False]
    kidneyUsed = n*[False]
    matching = n*[0]
    
    # Building the graph
    for k in range(1,n):
        graph[k]= P[k][0]
        
    # To print the progress
    step = 0
    if progress:
        print("\n\n=============START PROCESS=============\n\n")
        print("STEP : {}\nGraph      = {}\nAssigned   = {}\nKidneyUsed = {}\nMatching   = {}\n\n".format(step,graph,assigned,kidneyUsed,matching))
    
    # Initialisation    
    detectionResult = detect_cycle_wChains(graph,assigned,n)
    
    while detectionResult != (1,set()):
        if detectionResult[0] == 0: # Cycle
            action_cycle(graph,assigned,matching,kidneyUsed,detectionResult,n)
            if progress:
                print("DETECTED : Cycle\nRule : B\nCycle beginning : {}\n\n".format(detectionResult[1]))
        else:
            bestWChain = get_bestWChains_B(graph,assigned,detectionResult,n)
            action_wChain(graph,assigned,matching,kidneyUsed,bestWChain,n)
            if progress:
                print("DETECTED : w-Chain\nRule : B\nChain beginning : {}\nChain length : {}\n\n".format(bestWChain[0],bestWChain[1]))
        
        
        step += 1
        if progress:
            print("STEP : {}\nGraph      = {}\nAssigned   = {}\nKidneyUsed = {}\nMatching   = {}\n\n".format(step,graph,assigned,kidneyUsed,matching))
        
        
        detectionResult = detect_cycle_wChains(graph,assigned,n)
    
    
    if progress:
        print("==============END PROCESS==============\n\n\n\n")
    return matching

In [14]:
match_B(P,n)

[0, 9, 11, 2, 1, 7, 5, 6, 4, 0, 0, 3, 8]

### **Résultats**

In [15]:
print(match_A(P,n))
print(match_B(P,n))

[0, 9, 11, 2, 8, 7, 5, 6, 4, 0, 1, 3, 10]
[0, 9, 11, 2, 1, 7, 5, 6, 4, 0, 0, 3, 8]


### **Debuging**

In [16]:
#Exemple du sujet :
#Liste des reins non affectés préférés
#On affecte w à lui-même
graph = [0,9,11,2,5,3,3,6,6,3,11,3,11]
assigned = n*[False]
kidneyUsed = n*[False]
matching = n*[0]

In [17]:
print(detect_cycle_wChains(graph, assigned,n))

(0, 3)


In [18]:
comment = """
action_cycle(graph,assigned,matching,detect_cycle_wChains(graph, assigned,n),n)
print(matching)
print(graph)
#"""

In [19]:
comment = """
tuppleWChains = detect_cycle_wChains(graph, assigned,n)
bestWChain = get_bestWChains_A(graph,assigned,tuppleWChains,n)
action_wChain(graph,assigned,matching,bestWChain,n)
print(bestWChain)
print(matching)
print(graph)
#"""

In [20]:
comment = """
tuppleWChains = detect_cycle_wChains(graph, assigned,n)
bestWChain = get_bestWChains_B(graph,assigned,tuppleWChains,n)
action_wChain(graph,assigned,matching,bestWChain,n)
print(bestWChain)
print(matching)
print(graph)
#"""

## **Question 5**

In [21]:
mA = match_A(P,n,progress=True)
mB = match_B(P,n,progress=True)



=============START PROCESS=============


STEP : 0
Graph      = [0, 9, 11, 2, 5, 3, 3, 6, 6, 3, 11, 3, 11]
Assigned   = [False, False, False, False, False, False, False, False, False, False, False, False, False]
KidneyUsed = [False, False, False, False, False, False, False, False, False, False, False, False, False]
Matching   = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


DETECTED : Cycle
Rule : A
Cycle beginning : 3


STEP : 1
Graph      = [0, 9, 11, 2, 5, 7, 5, 6, 6, 0, 1, 3, 9]
Assigned   = [False, False, True, True, False, False, False, False, False, False, False, True, False]
KidneyUsed = [False, False, True, True, False, False, False, False, False, False, False, True, False]
Matching   = [0, 0, 11, 2, 0, 0, 0, 0, 0, 0, 0, 3, 0]


DETECTED : Cycle
Rule : A
Cycle beginning : 5


STEP : 2
Graph      = [0, 9, 11, 2, 9, 7, 5, 6, 4, 0, 1, 3, 9]
Assigned   = [False, False, True, True, False, True, True, True, False, False, False, True, False]
KidneyUsed = [False, False, True, True, False

## **Question 8**
On constate que 'ERROR !' n'est jamais imprimé, ce qui illustre la question 8.

In [22]:
P1 = [l.copy() for l in P] # Deep copy
matching = match_B(P,n)
for k in range(100):
    random.shuffle(P1[n-1])
    if P[n-1].index(match_B(P1,n)[n-1]) < P[n-1].index(matching[n-1]):
        print("ERROR !")


## **Question 9**

On commence par importe les données de l'énoncé

In [23]:
graph1 = np.array([[0,0,0,0,0,0,0,0,1,0,0,0],
[0,0,0,0,0,0,0,0,0,0,1,0],
[0,1,0,0,0,0,0,0,0,0,0,0],
[0,0,0,0,1,0,0,0,0,0,0,0],
[0,0,1,0,0,0,0,0,0,0,0,0],
[0,0,1,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,1,0,0,0,0,0,0],
[0,0,0,0,0,1,0,0,0,0,0,0],
[0,0,1,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,1,0],
[0,0,1,0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0,0,1,0]])

In [24]:
graph2 = np.array([[0,0,0,0,0,0,0,0,1,0,0,0],
[0,0,0,0,0,0,0,0,0,0,1,0],
[1,1,0,1,0,0,0,1,0,0,0,0],
[0,0,0,0,1,0,0,0,0,0,0,0],
[0,0,1,0,0,1,0,0,0,0,0,0],
[0,0,1,1,0,0,0,0,0,0,0,0],
[0,0,0,0,0,1,0,0,0,0,0,0],
[0,0,0,0,0,1,0,0,0,0,0,0],
[0,0,1,0,0,0,0,0,0,0,0,0],
[0,1,0,0,0,0,0,0,0,0,0,0],
[0,0,1,0,0,0,0,0,0,1,0,0],
[0,0,0,0,0,0,0,0,0,0,1,0]])

In [25]:
graph3 = np.array([[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,0,0,0,0,0,1,1,0,0,1,0,0,1,1,0,1],
[1,0,0,0,0,0,1,1,0,0,1,0,0,1,1,0,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,0,0,0,0,0,1,1,0,0,1,0,0,1,1,0,1],
[1,0,0,0,0,0,1,1,0,0,1,0,0,1,1,0,1],
[1,0,0,0,0,0,1,1,0,0,1,0,0,1,1,0,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,0,0,0,0,0,1,1,0,0,1,0,0,1,1,0,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],
[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]])

In [26]:
graph = [None,graph1,graph2,graph3]

Pour répondre à la question, on procède comme suit : pour chaque sommet, on effectue un parcours en largeur dont on borne la profondeur. On mémorise également les éléments par lesquels on est déjà passés. Puis, on execute cela pour tous les sommets du graphe.

Cette solution, simple à implémenter, à l'inconvénient de faire calculer plusieurs fois les mêmes choses. Pour accélérer les calculs, il faudrait mémoïzer tous les minimalInfeasiblePath  de longueur donnée partant d'un sommet donné, chose que l'on ne fait pas ici par manque de temps.

In [27]:
def get_unseen_neighbors(graph,seen,s):
    """Renvoie la liste des sommets voisins de s dans K et non vus dans seen."""
    n = len(graph)
    result = []
    for k in range(n):
        if graph[s,k] and not seen[k]:
            result.append(k)
    return result

In [28]:
def get_minimalInfeasible_fromSummit(graph,seen,depth,sStart):
    """Revoie la liste des minimalInfeasiblePath partant de sStart,
    de taille depth, et n'utilisant pas les sommets de seen."""
    if depth == 0:
        return [[]]
    
    if depth == 1:
        return [[sStart]]
    
    result = []
    for s in get_unseen_neighbors(graph,seen,sStart):
        seen[s] = True
        intermediateResult = get_minimalInfeasible_fromSummit(graph,seen,depth-1,s)
        for l in intermediateResult:
            result.append([sStart]+l)
        seen[s] = False
    
    return result
    

In [29]:
def get_minimalInfeasible(graph,depth):
    result = []
    n = len(graph)
    seen = n*[False]
    for sStart in range(n):
        seen[sStart] = True
        result = result + get_minimalInfeasible_fromSummit(graph,seen,depth,sStart)
        seen[sStart] = False
    return result

In [30]:
maxTime=0
for depth in range(6):
    for i in [1,2,3]:
        t = time()
        myResult = get_minimalInfeasible(graph[i],depth)
        delta = time() - t
        if delta > maxTime:
            print("Depth = {}, i = {}, CpuTime = {}".format(depth,i,delta))
            maxTime = delta


Depth = 0, i = 1, CpuTime = 7.152557373046875e-06
Depth = 0, i = 2, CpuTime = 9.059906005859375e-06
Depth = 2, i = 1, CpuTime = 4.410743713378906e-05
Depth = 2, i = 2, CpuTime = 5.1975250244140625e-05
Depth = 2, i = 3, CpuTime = 0.000164031982421875
Depth = 3, i = 3, CpuTime = 0.0027129650115966797
Depth = 4, i = 3, CpuTime = 0.037947893142700195
Depth = 5, i = 3, CpuTime = 0.6429641246795654
